In [1]:
from utils_c1 import *
from Model_c1 import *
from dataset_c1 import *
from train_c1 import *
from predict_c1 import *

In [2]:
# -------------------------------------------------------
#                   Electron Images
# -------------------------------------------------------
filename = "/DATA/Shashank/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
hf = h5py.File(filename, 'r')
X_Electron = np.array(hf["X"][:])
Y_Electron = np.array(hf["y"][:])
Y_Electron[0]

# -------------------------------------------------------
#                   Photon Images
# -------------------------------------------------------
filename1 = '/DATA/Shashank/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
h = h5py.File(filename1, 'r')
X_Photon = np.array(h["X"][:])
Y_Photon = np.array(h["y"][:])
Y_Photon[0]

0.0

In [3]:
X_train = np.concatenate((X_Electron, X_Photon), axis = 0)
X_train.shape

Y_train = np.concatenate((Y_Electron, Y_Photon))
Y_train.shape

(498000,)

In [4]:
import numpy as np

# Assuming X_train and Y_train are your data and labels arrays
combined = list(zip(X_train, Y_train))
np.random.shuffle(combined)
X_train_shuffled, Y_train_shuffled = zip(*combined)

# Convert back to numpy arrays if necessary
X_train_shuffled = np.array(X_train_shuffled)
Y_train_shuffled = np.array(Y_train_shuffled)

In [5]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
dataset = CustomDataset(X_train_shuffled, Y_train_shuffled, transform = transform)

In [ ]:
# import timm
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold
k_folds = 5
batch_size = 256
kf = KFold(n_splits=k_folds, shuffle=True)
trn_fold_loss = []
val_fold_loss = []
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    
    print(f'*** Fold {fold}***')
    print('Training Started.....')
    train_loader = DataLoader(dataset=dataset,
                              batch_size=batch_size,
                              sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )
    
    test_loader = DataLoader(dataset=dataset,
                             batch_size=batch_size,
                             sampler=torch.utils.data.SubsetRandomSampler(test_idx),
    )
    
    DEVICE = torch.device("cuda")
    model = Resnet_Ensemble(2, 15, BasicBlock, CBAM, resnet = ResNet)
    NUM_GPU = torch.cuda.device_count()
    if NUM_GPU > 1:
        model = nn.DataParallel(model)
    model = model.to(DEVICE)

    trainer = Trainer(fold, model, train_dataloader = train_loader, test_dataloader = test_loader)
    trn_loss, val_loss, trn_acc, val_acc = trainer.model_train(fold = fold, epochs = 1)
    trn_fold_loss.append(trn_loss)
    val_fold_loss.append(val_loss)

*** Fold 0***
Training Started.....


Epochs: 100%|██████████| 1/1 [03:27<00:00, 207.70s/epoch]

Epoch 1/1, Train Loss: 0.5966, Train Accuracy: 0.6908, Valid Loss: 0.6199, Valid Accuracy: 0.6732
*** Fold 1***
Training Started.....



Epochs: 100%|██████████| 1/1 [03:27<00:00, 207.71s/epoch]

Epoch 1/1, Train Loss: 0.5981, Train Accuracy: 0.6914, Valid Loss: 0.8125, Valid Accuracy: 0.6030
*** Fold 2***
Training Started.....


...

Epoch 1/1, Train Loss: 0.5959, Train Accuracy: 0.6891, Valid Loss: 0.5670, Valid Accuracy: 0.7154
*** Fold 3***
Training Started.....



Epochs: 100%|██████████| 1/1 [03:39<00:00, 219.93s/epoch]

Epoch 1/1, Train Loss: 0.5966, Train Accuracy: 0.6917, Valid Loss: 0.6199, Valid Accuracy: 0.6772


*** Fold 4***
Training Started.....


Epochs:   0%|          | 0/1 [00:00<?, ?epoch/s]

In [ ]:
for i in range(k_folds):
    plt.plot(trn_fold_loss[i], label='Training Loss')
    plt.plot(val_fold_loss[i], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()